## Install requirements

In [ ]:
!pip install fiona shapely pyproj rtree geopandas plotly pycountry networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 29.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=27ea51f1bbd5b164b4332552cf6d014ad8c2842cc9477b8de526347eb38a68ba
  Stored in directory: /root/.cache/pip/wheels/03/57/cc/290c5252ec97a6d78d36479a3c5e5ecc76318afcb241ad9dbe
Successfully built pycountry


## Load dependencies

In [ ]:
# notebook dependencies
import ipywidgets as widgets
from IPython.display import display
from google.colab import data_table
from google.colab import output
output.enable_custom_widget_manager()

# data handler libraries
import pandas as pd
import numpy as np
import geopandas as gp

# efficient queueing library
import heapq

# graph library
import networkx as nx

# visualisation libraries
import plotly.express as px
import plotly.graph_objects as go

# misc
import itertools
from urllib.request import urlretrieve
from tqdm import tqdm
import pycountry
import os

base_download_url = 'https://bbmaps.itu.int/geonetwork/srv/api/records/884b4f42-2d49-4622-b1d8-e6285bf46c7f/attachments/'

## Input parameters

In [ ]:
#@title
# Define the layout for the input fields
item_layout = widgets.Layout(
    width='auto',
    min_width='20px',
    flex='1 1 auto',
    display='flex',
    flex_flow='row wrap',
    align_items='center',  # Align items vertically in the middle
    justify_content='space-between'
)

item_layout2 = widgets.Layout(
    width='auto',
    min_width='20px',
    flex='1 1',
    display='flex',
    flex_flow='row wrap',
    align_items='center',  # Align items vertically in the middle
    justify_content='space-between'
)

# Create the input fields
country = widgets.Dropdown(
    description='Country:',
    options=list(map(lambda x: x.name, pycountry.countries)),
    value='Brazil',
    layout=item_layout2,
    style={'description_width': 'initial'}
)

municipality_code = widgets.Text(
    value='1504059',
    description='Municipaliy code:',
    layout=item_layout2,
    style={'description_width': 'initial'}
)

maximum_connection_length = widgets.FloatSlider(
    description='Maximum Connection Length:',
    min=0,
    max=100,
    step=1,
    value=15,
    layout=item_layout,
    style={'description_width': 'initial'}
)

# Create a container to hold the input fields
inputs_layout = widgets.VBox(
    children=[
        widgets.HBox([country], layout=widgets.Layout(justify_content='space-between')),
        widgets.HBox([municipality_code], layout=widgets.Layout(justify_content='space-between')),
        widgets.HBox([maximum_connection_length], layout=widgets.Layout(justify_content='space-between')),
        #widgets.HBox([max_tower_reach, n_visible], layout=widgets.Layout(justify_content='space-between'))
    ]
)

# Display the input fields
display(inputs_layout)


## Download & Initialize school dataset

In [ ]:
# Downloading school data from url
def download_entity_dataset(base_url, entity_type, country_name, administrative_division_code):
  """
  Downloads a dataset for a specific entity type and location.

  Args:
      base_url (str): The base URL where the dataset files are stored.
      entity_type (str): The type of entity to download the dataset for.
      country_name (str): The name of the country where the entity is located.
      administrative_division_code (str): The administrative division code for the entity.

  Returns:
      str: The filename of the downloaded dataset.

  Raises:
      pycountry.LookupError: If the country name is not recognized.

  """
  # Get the country code from the country name
  country_code = pycountry.countries.get(name = country.value).alpha_3

  # Construct the filename for the dataset
  entity_filename = f'{country_code.lower()}_{administrative_division_code}_{entity_type}.parquet'

  # Download the dataset file
  urlretrieve(base_url + entity_filename, entity_filename)

  # Return the filename of the downloaded dataset
  return entity_filename


# Reading school data from a file into a python data table
def read_file(file_path):
    """
    Read a file from the data directory.

    Args:
        file_name (str): The name of the file to read.

    Returns:
        pd.DataFrame or gp.GeoDataFrame: The data read from the file.
    """
    suffix = os.path.splitext(file_path)[1]

    assert os.path.exists(file_path), f'{file_path} cannot be found in the data directory!'

    if suffix == '.csv':
        return pd.read_csv(file_path)
    elif suffix == '.xlsx' or suffix == '.xls':
        return pd.read_excel(file_path, engine = 'openpyxl')
    elif suffix == '.shp' or suffix == '.zip':
        return gp.read_file(file_path)
    elif suffix == '.parquet':
        try:
          return gp.read_parquet(file_path)
        except:
          return pd.read_parquet(file_path)
    elif suffix == '.geoparquet':
        return gp.read_parquet(file_path)
    elif suffix == '.json':
        return pd.read_json(file_path)
    elif suffix == '.geojson':
        return gp.read_file(file_path)
    elif suffix == '.gpkg':
        return gp.read_file(file_path)
    else:
        raise ValueError(f"Unsupported file type: {suffix}")

In [ ]:
file_name_school = download_entity_dataset(base_download_url, 'school', country.value, municipality_code.value)

In [ ]:
df_school = read_file(file_name_school)
data_table.DataTable(df_school, num_rows_per_page=10)

,lat,lon,label,geometry
source_id,,,,
15092518,-2.006472,-47.454827,unconnected_school,POINT (-47.45483 -2.00647)
15092526,-2.042973,-47.557938,connected_school,POINT (-47.55794 -2.04297)
15092534,-1.915732,-47.520132,connected_school,POINT (-47.52013 -1.91573)
15092542,-2.040823,-47.558717,connected_school,POINT (-47.55872 -2.04082)
15092577,-2.042782,-47.551758,connected_school,POINT (-47.55176 -2.04278)
15092585,-2.045535,-47.551423,connected_school,POINT (-47.55142 -2.04554)
15092593,-2.040445,-47.563479,connected_school,POINT (-47.56348 -2.04045)
15092640,-1.903420,-47.485589,unconnected_school,POINT (-47.48559 -1.90342)
15092690,-1.891456,-47.477476,unconnected_school,POINT (-47.47748 -1.89146)


## Download & Initialize fiber node dataset

In [ ]:
file_name_fibernode = download_entity_dataset(base_download_url, 'fibernode', country.value, municipality_code.value)

In [ ]:
df_fibernode = read_file(file_name_fibernode)
data_table.DataTable(df_fibernode, num_rows_per_page=10)

,lat,lon,label,geometry
source_id,,,,
fiber_1615,-1.915700,-47.520100,fiber_node,POINT (-47.52010 -1.91570)
fiber_2011,-2.045600,-47.565300,fiber_node,POINT (-47.56530 -2.04560)
fiber_2060,-2.051000,-47.544400,fiber_node,POINT (-47.54440 -2.05100)
fiber_2072,-2.054000,-47.543800,fiber_node,POINT (-47.54380 -2.05400)
fiber_38240,-1.870090,-47.524109,fiber_node,POINT (-47.52411 -1.87009)
...,...,...,...,...
fiber_38324,-2.062238,-47.546082,fiber_node,POINT (-47.54608 -2.06224)
fiber_38325,-2.062238,-47.540588,fiber_node,POINT (-47.54059 -2.06224)
fiber_38326,-2.056748,-47.535095,fiber_node,POINT (-47.53510 -2.05675)


#### Concatenate school and fibernode dataset

In [ ]:
# concatenate school and fiber node data
df_locations = pd.concat([df_school, df_fibernode])

# extract connected index
connected_idx = df_locations.loc[df_locations.label != 'unconnected_school'].index
unconnected_idx = df_locations.loc[df_locations.label == 'unconnected_school'].index

## Create Line of Sight (LOS) Graph

In [ ]:
def haversine_(lats, lons, R = 6371.0, upper_tri = False):

    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees) using the
    Haversine formula.

    Parameters
    ----------
    lats, lons: array-like
        Arrays of latitudes and longitudes of the two points.
        Each array should have shape (2,) where the first element
        is the latitude and the second element is the longitude.
    upper_tri : bool, optional
        If True, returns the distance matrix in upper triangular form.
        Default is False.
    R : float, optional
        Radius of the earth in kilometers. Default is 6371.0 km.

    Returns
    -------
    ndarray
        The distance matrix between the points in kilometers.
        If `upper_tri` is True, returns the upper triangular form of the matrix.

    """

    # Convert latitudes and longitudes to radians
    lat_rads = np.radians(lats)
    lon_rads = np.radians(lons)

    # Compute pairwise haversine distances using broadcasting
    dlat = lat_rads[:, np.newaxis] - lat_rads[np.newaxis, :]
    dlon = lon_rads[:, np.newaxis] - lon_rads[np.newaxis, :]
    a = np.sin(dlat / 2) ** 2 + np.cos(lat_rads[:, np.newaxis]) * np.cos(lat_rads[np.newaxis, :]) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distances = R * c

    if upper_tri:
        i_upper = np.triu_indices(distances.shape[0], k=1)
        distances = distances[i_upper]

    return distances


def generate_all_index_pairs(vals):
    """
    Generates all possible pairs of indices from a list of values.

    Args:
        vals (list): The list of values.

    Returns:
        tuple: A tuple containing two lists, where the first list represents the original nodes
               and the second list represents the destination nodes.

    """
    # Generate all combinations of indices from the list of values
    ind = itertools.combinations(vals, 2)

    # Unzip the combinations into separate lists for original nodes and destination nodes
    orig_nodes, dest_nodes = zip(*ind)

    # Return the lists of original nodes and destination nodes
    return orig_nodes, dest_nodes


def create_line_of_sight_graph(locations):
    """
    Creates a line of sight graph based on the locations.

    Args:
        locations (DataFrame): A DataFrame containing location data.

    Returns:
        networkx.Graph: The line of sight graph.

    """
    # Generate all index pairs for origin and destination nodes
    orig_nodes, dest_nodes = generate_all_index_pairs(locations.index)

    # Compute geodesic distances for all index pairs using haversine formula
    distances = haversine_(locations.lat.to_numpy(), locations.lon.to_numpy(), upper_tri=True) * 1000

    # Initialize a networkx graph
    graph = nx.Graph()

    # Add weighted edges to the graph using the original nodes, destination nodes, and distances
    graph.add_weighted_edges_from(zip(orig_nodes, dest_nodes, distances))

    # Return the line of sight graph
    return graph

In [ ]:
graph = create_line_of_sight_graph(df_locations)

## Connect unconnected schools

In [ ]:
def connect_schools(locations, connected_idx, max_connection_length, graph_):

    """
    Connects unconnected schools to the nearest connected school dynamically using the given graph.
    Returns a dictionary containing the fiber path information for each school.

    Args:
    locations : DataFrame of all locations where the index is the location identifier.
    connected_idx: List of index that are either fiber node or point connected with fiber.
    max_connection_length: Maximum length of cable that can be used to connect unconnected location.
    graph_ : Networkx graph object representing the line of sight graph.

    Returns:
    fiber_path_dict (dict): Dictionary containing the fiber path and other information for each school that is connected to the network.
                            Keys are school/node indices, values are dictionaries containing:
                            - closest_node_id: ID of the closest fiber node to the school.
                            - closest_node_distance: Distance between the school and the closest fiber node.
                            - connected_node_id: ID of the fiber node to which the school is connected.
                            - connected_node_distance: Distance between the school and the fiber node to which it is connected.
                            - upstream_node_id: ID of the upstream connected node in the network.
                            - upstream_node_distance: Distance between the school and the upstream connected node.
                            - fiber_path: List of node IDs representing the fiber path from the connected node to the closest fiber node to the school.
    """


    # Get sets of connected and unconnected school indices
    connected_idx_cache = connected_idx
    connected_idx = set(connected_idx)
    unconnected_idx = set(locations.index) - connected_idx

    # Generate combinations of unconnected and connected school indices
    ind = itertools.product(unconnected_idx, connected_idx)
    orig_nodes, dest_nodes = zip(*ind)

    # Compute shortest path lengths between each pair of unconnected and connected schools
    sp_lengths = []
    for orig_node, dest_node in zip(orig_nodes, dest_nodes):
      sp_lengths = sp_lengths + [nx.shortest_path_length(graph_, orig_node, dest_node, weight = 'weight')]

    # Compute distances from each unconnected school to the closest connected school
    dist_to_connected = np.array(sp_lengths).reshape(len(unconnected_idx), len(connected_idx))
    closest_connected_id = connected_idx_cache[np.argmin(dist_to_connected, axis=1)]
    closest_connected_distance = np.min(dist_to_connected, axis=1)

    # Create a dictionary to store the fiber path information for each school
    fiber_path_dict = dict.fromkeys(locations.index)

    # Initialize the fiber path information for each connected school
    fiber_path_dict.update({idx:
                            dict(
                                closest_node_id = idx,
                                closest_node_distance = 0,
                                connected_node_id = idx,
                                connected_node_distance = 0,
                                fiber_path = []
                            ) for idx in connected_idx})

    # Initialize the fiber path information for each unconnected school
    fiber_path_dict.update([(idx,
                dict(
                    closest_node_id = closest_connected_id[i],
                    closest_node_distance = closest_connected_distance[i],
                    connected_node_id = '',
                    connected_node_distance =0,
                    fiber_path = []
                )
            ) for i, idx in enumerate(unconnected_idx)])

    # Create a priority queue to store candidate fiber connections
    queue = []

    # Add all pairs of unconnected and connected schools with distances below the max connection length to the queue
    for d_ in zip(sp_lengths, orig_nodes, dest_nodes):
        if d_[0] <= max_connection_length:
            heapq.heappush(queue, d_)

    # Iterate over the candidate fiber connections until all unconnected schools are connected
    while queue:
        # Pop the fiber connection with the smallest distance
        min_dist, min_node, upstream_node = heapq.heappop(queue)

        # Skip the connection if the minimum node is already connected
        if min_node in connected_idx:
            continue

        # Add the minimum node to the connected set if the distance is below the max connection length
        if min_dist <= max_connection_length:
            connected_idx.add(min_node)
            unconnected_idx.remove(min_node)

            # Update the fiber path information for the minimum node
            fiber_path_dict[min_node].update(
                connected_node_id = fiber_path_dict[upstream_node]['connected_node_id'],
                connected_node_distance = '',
                upstream_node_id = upstream_node,
                upstream_node_distance = min_dist,
                fiber_path = fiber_path_dict[upstream_node]['fiber_path'] + [upstream_node]
            )

            # Exit the loop if all schools are connected
            if len(unconnected_idx) == 0:
                break

            # Add new candidate fiber connections to the queue for the newly connected node
            ind = itertools.product(unconnected_idx, [min_node])
            orig_nodes, dest_nodes = zip(*ind)

            new_sp_lengths = []
            for orig_node, dest_node in zip(orig_nodes, dest_nodes):
              new_sp_lengths = new_sp_lengths + [nx.shortest_path_length(graph, orig_node, dest_node, weight = 'weight')]

            for d_ in zip(new_sp_lengths, orig_nodes, dest_nodes):
                if d_[0] <= max_connection_length:
                    heapq.heappush(queue, d_)

    return fiber_path_dict, connected_idx



In [ ]:
fiber_path_dict, fiber_path_connected_idx = connect_schools(df_locations, connected_idx, maximum_connection_length.value*1000, graph)

In [ ]:
data_table.DataTable(pd.DataFrame(fiber_path_dict.values(), fiber_path_dict.keys()), num_rows_per_page=25)

,closest_node_id,closest_node_distance,connected_node_id,connected_node_distance,fiber_path,upstream_node_id,upstream_node_distance
15092518,fiber_38245,4479.12635,fiber_38328,,"[fiber_38328, 15552691]",15552691,3793.977927
15092526,15092526,0.00000,15092526,0,[],NaN,NaN
15092534,15092534,0.00000,15092534,0,[],NaN,NaN
15092542,15092542,0.00000,15092542,0,[],NaN,NaN
15092577,15092577,0.00000,15092577,0,[],NaN,NaN
...,...,...,...,...,...,...,...
fiber_38324,fiber_38324,0.00000,fiber_38324,0,[],NaN,NaN
fiber_38325,fiber_38325,0.00000,fiber_38325,0,[],NaN,NaN
fiber_38326,fiber_38326,0.00000,fiber_38326,0,[],NaN,NaN
fiber_38327,fiber_38327,0.00000,fiber_38327,0,[],NaN,NaN


## Visualise fiber path



In [ ]:
mapbox_token = ''

> The following cell contains plot function definition. You have to run it befor executing the code in the next cells. Also, please do not forget to put your mapbox access token into above cell.

In [ ]:
#@title
def plot_locations(locations, zoom = 10):

    config = dict(
        fiber_node = dict(
            label = 'Fiber node',
            marker = dict(
                color = 'rgba(139, 212, 50,1)',
            )
        ),
        connected_school = dict(
            label = 'Connected school',
            marker = dict(
                color = 'rgba(84, 111, 194,1)',
            )
        ),
        unconnected_school = dict(
            label = 'Unconnected school',
            marker = dict(
                color = 'rgba(255, 97, 91, 1)',
            )
        ),
        celltower = dict(
            label = 'Cell tower',
            marker = dict(
                color = 'rgba(255, 201, 61, 1)'
            )
        )
    )

    fig = go.Figure()

    for label in locations.label.unique():
        df = locations.loc[locations.label == label]

        fig.add_trace(
            go.Scattermapbox(
                lat = df.lat,
                lon = df.lon,
                customdata=np.stack((df.index, list(map(lambda x: config[x]['label'], df.label))), axis=-1),
                mode = 'markers',
                name = config[label]['label'],
                visible = True,
                marker = config[label]['marker'],
                hoverinfo = 'text',
                hovertemplate = '<b>Source ID:</b> %{customdata[0]}<br>'
                                '<b>Label:</b> %{customdata[1]}'
                                '<extra></extra>'
            )
        )
    #   General Map Styles
    fig.update_layout(
        margin = dict(l=10, r=10, t=10, b=10, pad=5),
        plot_bgcolor = '#161515',
        paper_bgcolor = '#161515',
        clickmode = 'event+select',
        hovermode = 'closest',
        showlegend = True,
        legend = dict(
            x = 0,
            y = 1,
            font = dict(
                color = 'white'
            ),
            orientation = 'v',
            bgcolor = 'rgba(0,0,0,0)'
        ),
        mapbox = go.layout.Mapbox(
            accesstoken = mapbox_token,
            bearing = 0,
            center=go.layout.mapbox.Center(
                lat=locations.lat.mean(), lon=locations.lon.mean()
            ),
            pitch = 5,
            zoom = zoom,
            style = 'mapbox://styles/utkucanozturk/clftvbap6005i01mz85zne853',
        ),
    )

    return fig

In [ ]:
fig = plot_locations(df_locations)

In [ ]:
for loc_ in pd.DataFrame(fiber_path_dict.values(), fiber_path_dict.keys()).itertuples():
  if loc_.upstream_node_id == loc_.upstream_node_id:
    loc_data = df_locations.loc[loc_.Index]
    ups_node_data = df_locations.loc[loc_.upstream_node_id]

    fig.add_trace(
        go.Scattermapbox(
            mode='lines',
            lat=[loc_data.lat, ups_node_data.lat],
            lon=[loc_data.lon, ups_node_data.lon],
            customdata = np.stack(([loc_data.name], [ups_node_data.name], [loc_.upstream_node_distance]), axis = -1),
            line=dict(width=2, color='orange'),
            showlegend = False,
            #name= f'{loc_data.name}-{ups_node_data.name}: {loc_.upstream_node_distance}',
            hoverinfo = 'text',
            hovertemplate = '<b>u:</b> %{customdata[0]}<br>'
                            '<b>v:</b> %{customdata[1]}<br>'
                            '<b>Length:</b> %{customdata[2]}'
                            '<extra></extra>'
        )
    )
fig